<a href="https://colab.research.google.com/github/VardanDavtyan/ML-DL/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Flatten, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gensim

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
validation_data = pd.read_csv('./twitter_validation.csv')
training_data = pd.read_csv('./twitter_validation.csv')

In [ ]:
validation_data

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...,...,...
994,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
995,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
996,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
997,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [ ]:
training_data

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...,...,...
994,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
995,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
996,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
997,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [ ]:
variants = ['Positive', 'Negative', 'Neutral', 'Irrelevant']
def get_indexes(text):
  d = None
  if text == variants[0]:
    d = [1, 0, 0, 0]
  elif text == variants[1]:
    d = [0, 1, 0, 0]
  elif text == variants[2]:
    d = [0, 0, 1, 0]
  elif text == variants[3]:
    d = [0, 0, 0, 1]
  return np.array(d)

get_indexes('Negative')

array([0, 1, 0, 0])

In [ ]:
def normalize_data(data):

  data = pd.DataFrame(data)
  data.drop(data.columns[[0, 1]], axis=1, inplace=True)
  data = data[data.columns[::-1]]

  data.rename(columns={data.columns[0]: 'data', data.columns[1]: 'value_text'},inplace=True)

  l = []
  for i in range(len(data.index)):
    l.append(get_indexes(data.loc[i, 'value_text']))
  data.insert(0, 'value', l)

  data = data[data.columns[::-1]]
  data.drop(data.columns[[0]], axis=1, inplace=True)

  return data

In [ ]:
training_data = normalize_data(training_data)
validation_data = normalize_data(validation_data)

In [ ]:
maxWordsCount = 4886
max_text_len = 10
dim = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
sentences = training_data['data'].to_list()

In [ ]:
def preprocess_data(data):
  tokenizer.fit_on_texts(data['data'])
  data['data'] = tokenizer.texts_to_sequences(data['data'])
  data['data'] = pad_sequences(data['data'], maxlen=max_text_len)

  X = np.expand_dims(data['data'].to_list(), axis=-1)
  Y = np.array(data['value'].to_list())

  return (X, Y)

In [ ]:
x_train, y_train = preprocess_data(training_data)
x_test, y_test = preprocess_data(validation_data)

In [ ]:

w2v_model = gensim.models.Word2Vec(sentences=sentences, vector_size=dim, window=10, min_count=1)
vocab_size = len(tokenizer.word_index) + 1
vocab = tokenizer.word_index

In [ ]:
vocab_size

4887

In [ ]:


def get_weight_matrix(model):
  weight_matrix = np.zeros((vocab_size, dim))
  for word, i in vocab.items():
    if word in model.wv:
      weight_matrix[i] = model.wv[word]

  return weight_matrix

embedding_vectors = get_weight_matrix(w2v_model)

In [ ]:
model  = Sequential()
model.add(Embedding(vocab_size, dim, weights=[embedding_vectors], input_length=max_text_len))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dense(4, activation="sigmoid"))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 10, 1000)          4887000   
                                                                 
 gru_4 (GRU)                 (None, 10, 128)           433920    
                                                                 
 gru_5 (GRU)                 (None, 64)                37248     
                                                                 
 dense_4 (Dense)             (None, 4)                 260       
                                                                 
Total params: 5,358,428
Trainable params: 5,358,428
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
8/8 [==============================] - 7s 233ms/step - loss: 1.3857 - accuracy: 0.2773 - val_loss: 1.3833 - val_accuracy: 0.4705
Epoch 2/100
8/8 [==============================] - 1s 92ms/step - loss: 1.3823 - accuracy: 0.4815 - val_loss: 1.3802 - val_accuracy: 0.5445
Epoch 3/100
8/8 [==============================] - 1s 102ms/step - loss: 1.3792 - accuracy: 0.5475 - val_loss: 1.3774 - val_accuracy: 0.5546
Epoch 4/100
8/8 [==============================] - 1s 132ms/step - loss: 1.3764 - accuracy: 0.5465 - val_loss: 1.3745 - val_accuracy: 0.5506
Epoch 5/100
8/8 [==============================] - 1s 126ms/step - loss: 1.3736 - accuracy: 0.5465 - val_loss: 1.3715 - val_accuracy: 0.5566
Epoch 6/100
8/8 [==============================] - 1s 142ms/step - loss: 1.3703 - accuracy: 0.5546 - val_loss: 1.3682 - val_accuracy: 0.5566
Epoch 7/100
8/8 [==============================] - 1s 94ms/step - loss: 1.3670 - accuracy: 0.5546 - val_loss: 1.3645 - val_accuracy: 0.5596
Epoch 8/100
8/8

In [ ]:
def preprocess_text(text):
  text = tokenizer.texts_to_sequences([text])
  text = pad_sequences(text, maxlen=max_text_len)
  return text

In [ ]:
text = input()
res = model.predict(preprocess_text(text))
print(res, variants[np.argmax(res, axis=-1)[0]], sep='\n')